# 2.3. [**Partitionnement**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/2_3_clustering.ipynb)</br>([*Clustering*](https://scikit-learn.org/stable/modules/clustering.html))

Le [**partitionnement de données**](https://en.wikipedia.org/wiki/Cluster_analysis) (*clustering*) non étiquetées peut être effectué avec le module [**`sklearn.cluster`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).

Chaque algorithme de clustering se décline en deux variantes : une classe, qui implémente la méthode `fit` pour découvrir les clusters sur les données d'entraînement, et une fonction, qui, compte tenu des données d'entraînement, renvoie un tableau d'étiquettes entières correspondant aux différents clusters. Pour la classe, les étiquettes sur les données d'apprentissage se trouvent dans l'attribut `labels_`.

**Données d'entrée**

Une chose importante à noter est que les algorithmes implémentés dans ce module peuvent prendre différents types de matrices en entrée. Toutes les méthodes acceptent des matrices de données standard de forme `(n_samples, n_features)`. Celles-ci peuvent être obtenues à partir des classes du module [**`sklearn.feature_extraction`**](https://scikit-learn.org/stable/modules/clustering.html). Pour [**`AffinityPropagation`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html), [**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) et [**`DBSCAN`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html), on peut également entrer des matrices de similarité de forme `(n_samples, n_samples)`. Celles-ci peuvent être obtenues à partir des fonctions du module [**`sklearn.metrics.pairwise`**](https://scikit-learn.org/stable/modules/classes.html).

## 2.3.1. **Vue d'ensemble des méthodes de partitionnement**<br/>([*Overview of clustering methods*](https://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods))

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_001.png)

|Nom de la méthode|Paramètres|Évolutivité|Cas d'utilisation|Géométrie (métrique utilisée)|
|-|-|-|-|-|
|[**K-Means** (2.3.2)](https://scikit-learn.org/stable/modules/clustering.html#k-means)|nombre de clusters|`n_samples` très grand, `n_clusters` moyen avec [**code MiniBatch** (2.3.2.2)](https://scikit-learn.org/stable/modules/clustering.html#mini-batch-kmeans)|Usage général, même taille de cluster, géométrie plate, pas trop de clusters, inductif|Distances entre les points|
|[**Propagation par affinité** (2.3.3)](https://scikit-learn.org/stable/modules/clustering.html#affinity-propagation)|amortissement, préférence d'échantillon|Non évolutif avec `n_samples`|Nombreux clusters, taille de cluster inégale, géométrie non plate, inductif|Distance du graphe (par exemple, graphe du plus proche voisin)|
|[**Déplacement moyen** (2.3.4)](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)|bande passante|Non évolutif avec `n_samples`|De nombreux clusters, taille de cluster inégale, géométrie non plate, inductif|Distances entre les points|
|[**Regroupement spectral** (2.3.5)](https://scikit-learn.org/stable/modules/clustering.html#spectral-clustering)|nombre de clusters|`n_samples` moyen, petit `n_clusters`|Peu de clusters, même taille de cluster, géométrie non plate, transductif|Distance du graphe (par exemple, graphe du plus proche voisin)|
|[**Regroupement hiérarchique** (2.3.6)](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)|nombre de clusters ou seuil de distance|Grands `n_samples` et `n_clusters`|De nombreux clusters, éventuellement des contraintes de connectivité, transductif|Distances entre les points|
|[**Partitionnement agglomératif** (2.3.6)](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering)|nombre de clusters ou seuil de distance, type de liaison, distance|Grands `n_samples` et `n_clusters`|De nombreux clusters, éventuellement des contraintes de connectivité, des distances non euclidiennes, transductif|Toute distance par paire|
|[**DBSCAN** (2.3.7)](https://scikit-learn.org/stable/modules/clustering.html#dbscan)|taille du voisinage|`n_samples` très grand, `n_clusters` moyen|Géométrie non plate, tailles de cluster inégales, suppression des valeurs aberrantes, transductif|Distances entre les points les plus proches|
|[**OPTICS** (2.3.8)](https://scikit-learn.org/stable/modules/clustering.html#optics)|appartenance minimale au cluster|Très grand `n_samples`, grand `n_clusters`|Géométrie non plate, tailles de cluster inégales, densité de cluster variable, suppression des valeurs aberrantes, transductif|Distances entre les points|
|[**Mélanges gaussiens** (2.1)](https://scikit-learn.org/stable/modules/mixture.html#mixture)|beaucoup|Non évolutif|Géométrie plate, bonne pour l'estimation de la densité, inductif|Distances de Mahalanobis aux centres|
|[**BIRCH** (2.3.9)](https://scikit-learn.org/stable/modules/clustering.html#birch)|facteur de ramification, seuil, clustereur global facultatif|Grands `n_clusters` et `n_samples`|Grand ensemble de données, suppression des valeurs aberrantes, réduction des données, inductif|Distance euclidienne entre les points|
|[**Bisecting K-Means** (2.3.6.5)](https://scikit-learn.org/stable/modules/clustering.html#bisect-k-means)|nombre de clusters|`n_samples` très grand, `n_clusters` moyen|Usage général, même taille de cluster, géométrie plate, pas de clusters vides, inductif, hiérarchique|Distances entre les points|

Le clustering de géométrie non plate est utile lorsque les clusters ont une forme spécifique, c'est-à-dire une variété non plate, et que la distance euclidienne standard n'est pas la bonne métrique. Ce cas se présente dans les deux rangées supérieures de la figure ci-dessus.

Les modèles de mélange gaussiens, utiles pour le clustering, sont décrits dans un [**autre chapitre de la documentation dédié aux modèles de mélange** (2.1)](https://scikit-learn.org/stable/modules/mixture.html#mixture). KMeans peut être considéré comme un cas particulier de modèle de mélange gaussien avec une covariance égale par composant.

Les méthodes de regroupement [**transductives**](https://scikit-learn.org/stable/glossary.html#term-transductive) (contrairement aux méthodes de regroupement [**inductives**](https://scikit-learn.org/stable/glossary.html#term-inductive)) ne sont pas conçues pour être appliquées à de nouvelles données invisibles.

## 2.3.2. **K-moyennes**<br/>([*K-means*](https://scikit-learn.org/stable/modules/clustering.html#k-means))

L'algorithme [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) regroupe les données en essayant de séparer les échantillons en $n$ groupes de variance égale, en minimisant un critère connu sous le nom d'inertie ou somme des carrés intra-cluster (voir ci-dessous). Cet algorithme nécessite que le nombre de clusters soit spécifié. Il s'adapte bien à un grand nombre d'échantillons et a été utilisé dans un large éventail de domaines d'application dans de nombreux domaines.

L'algorithme des k-moyennes divise un ensemble de $N$ échantillons en $K$ grappes $C$ disjointes, chacune décrite par la moyenne $\mu_j$ des échantillons de la grappe. Les moyennes sont communément appelées les "barycentres" (*centroïdes*) du cluster; noter qu'elles ne sont pas, en général, des points de $X$, bien qu'elles partagent le même espace.

L'algorithme K-means vise à choisir des centroïdes qui minimisent l'**inertie**, ou **critère de somme des carrés intra-cluster** :

$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$$

L'inertie peut être reconnue comme une mesure de la cohérence interne des clusters. Elle souffre de divers inconvénients :

L'inertie peut être reconnue comme une mesure de la cohérence interne des clusters. Il souffre de divers inconvénients :
* L'inertie fait l'hypothèse que les clusters sont convexes et isotropes, ce qui n'est pas toujours le cas. Il répond mal aux grappes allongées ou aux variétés aux formes irrégulières.
* L'inertie n'est pas une métrique normalisée : nous savons simplement que les valeurs inférieures sont meilleures et que zéro est optimal. Mais dans les espaces de très grande dimension, les distances euclidiennes ont tendance à gonfler (c'est un exemple de ce qu'on appelle la "malédiction de la dimensionnalité"). L'exécution d'un algorithme de réduction de dimensionnalité tel que l'[**analyse en composantes principales (ACP)** (2.5.1)](https://scikit-learn.org/stable/modules/decomposition.html#pca) avant le clustering k-means peut atténuer ce problème et accélérer les calculs.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_assumptions_001.png)

Le K-means est souvent appelé algorithme de Lloyd. En termes simples, l'algorithme comporte trois étapes. La première étape consiste à choisir les centroïdes initiaux, la méthode la plus élémentaire étant de choisir $k$ échantillons dans l'ensemble de données $X$. Après initialisation, K-means consiste à boucler entre les deux autres étapes. La première étape affecte chaque échantillon à son centroïde le plus proche. La deuxième étape crée de nouveaux centroïdes en prenant la valeur moyenne de tous les échantillons attribués à chaque centroïde précédent. La différence entre l'ancien et le nouveau barycentre est calculée et l'algorithme répète ces deux dernières étapes jusqu'à ce que cette valeur soit inférieure à un seuil. En d'autres termes, il se répète jusqu'à ce que les centroïdes ne bougent pas de manière significative.

K-means équivaut à l'algorithme de maximisation des attentes avec une petite matrice de covariance diagonale entièrement égale.

L'algorithme peut également être compris à travers le concept de [**diagrammes de Voronoi** (wkpd)](https://en.wikipedia.org/wiki/Voronoi_diagram). Tout d'abord, le diagramme de Voronoi des points est calculé à l'aide des centroïdes actuels. Chaque segment du diagramme de Voronoi devient un cluster distinct. Deuxièmement, les centroïdes sont mis à jour à la moyenne de chaque segment. L'algorithme répète ensuite ceci jusqu'à ce qu'un critère d'arrêt soit rempli. Habituellement, l'algorithme s'arrête lorsque la diminution relative de la fonction objectif entre les itérations est inférieure à la valeur de tolérance donnée. Ce n'est pas le cas dans cette implémentation : l'itération s'arrête lorsque les centroïdes se déplacent moins que la tolérance.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kmeans_digits_001.png)

Avec suffisamment de temps, les K-moyennes convergeront toujours, mais cela peut être à un minimum local. Cela dépend fortement de l'initialisation des centroïdes. De ce fait, le calcul est souvent effectué plusieurs fois, avec des initialisations différentes des barycentres. Une méthode pour aider à résoudre ce problème est le schéma d'initialisation k-means++, qui a été implémenté dans scikit-learn (utilisez le paramètre `init='k-means++'`). Cela initialise les centroïdes pour qu'ils soient (généralement) distants les uns des autres, ce qui donne probablement de meilleurs résultats que l'initialisation aléatoire, comme indiqué dans la référence.

K-means++ peut également être appelé indépendamment pour sélectionner des graines pour d'autres algorithmes de clustering, voir [**`sklearn.cluster.kmeans_plusplus`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.kmeans_plusplus.html) pour plus de détails et un exemple d'utilisation.

L'algorithme prend en charge les poids d'échantillon, qui peuvent être donnés par un paramètre `sample_weight`. Cela permet d'attribuer plus de poids à certains échantillons lors du calcul des centres de cluster et des valeurs d'inertie. Par exemple, attribuer un poids de 2 à un échantillon équivaut à ajouter un doublon de cet échantillon à l'ensemble de données $X$.

K-means peut être utilisé pour la quantification vectorielle. Ceci est réalisé en utilisant la méthode de transformation d'un modèle entraîné de [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

### 2.3.2.1 **Parallélisme de bas niveau**<br/>([*Low-level parallelism*](https://scikit-learn.org/stable/modules/clustering.html#low-level-parallelism))

[**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) bénéficie du parallélisme basé sur OpenMP via Cython. De petits morceaux de données (256 échantillons) sont traités en parallèle, ce qui en plus produit une faible empreinte mémoire. Pour plus de détails sur la façon de contrôler le nombre de threads, veuillez vous référer à nos notes sur le [**parallélisme** (8.3.1)](https://scikit-learn.org/stable/computing/parallelism.html#parallelism).

#### Exemples

##### [**Démonstration des hypothèses de k-means**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_kmeans_assumptions.ipynb)<br/>([*Demonstration of k-means assumptions*](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html))

Démontrer quand k-means fonctionne intuitivement et quand ce n'est pas le cas.

##### [**Une démonstration du clustering K-Means sur les données des chiffres manuscrits**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_kmeans_digits.ipynb)<br/>([*A demo of K-Means clustering on the handwritten digits data*](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html))

Clustering des chiffres manuscrits.

#### Références

[?] Arthur, David and Sergei Vassilvitskii [“**k-means++: The advantages of careful seeding**](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf)[”](https://drive.google.com/file/d/11vuxCwPrrjuv8JakiEF5TCYyMBZzqoVg/view?usp=share_link), Actes du dix-huitième symposium annuel ACM-SIAM sur les algorithmes discrets, Society for Industrial and Applied Mathematics (2007)

### 2.3.2.2. **Mini lot K-moyennes**<br/>([Mini Batch K-Means](https://scikit-learn.org/stable/modules/clustering.html#mini-batch-k-means))

Le [**`MiniBatchKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) est une variante de l'algorithme [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) qui utilise des mini-batchs pour réduire le temps de calcul, tout en essayant d'optimiser la même fonction objectif. Les mini-lots sont des sous-ensembles des données d'entrée, échantillonnés de manière aléatoire à chaque itération d'apprentissage. Ces mini-lots réduisent considérablement la quantité de calculs nécessaires pour converger vers une solution locale. Contrairement à d'autres algorithmes qui réduisent le temps de convergence des k-means, les mini-batch k-means produisent des résultats qui ne sont généralement que légèrement moins bons que l'algorithme standard.

L'algorithme itère entre deux étapes principales, similaires au k-means vanille ??. Dans la première étape, $b$ échantillons sont tirés au hasard de l'ensemble de données, pour former un mini-lot. Ceux-ci sont ensuite affectés au centroïde le plus proche. Dans la deuxième étape, les centroïdes sont mis à jour. Contrairement aux k-moyennes, cela se fait sur une base par échantillon. Pour chaque échantillon du mini-lot, le centroïde attribué est mis à jour en prenant la moyenne en continu de l'échantillon et de tous les échantillons précédents attribués à ce centroïde. Cela a pour effet de diminuer le taux de variation d'un centroïde au fil du temps. Ces étapes sont exécutées jusqu'à ce qu'une convergence ou un nombre prédéterminé d'itérations soit atteint.

[**`MiniBatchKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) converge plus rapidement que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), mais la qualité des résultats est réduite. En pratique, cette différence de qualité peut être assez faible, comme le montrent l'exemple et la référence citée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_mini_batch_kmeans_001.png)

#### Exemples

##### [**Comparaison des algorithmes de clustering K-Means et MiniBatchKMeans**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_mini_batch_kmeans.ipynb)<br/>([*Comparison of the K-Means and MiniBatchKMeans clustering algorithms*](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html))

Comparaison de KMeans et MiniBatchKMeans.

##### [**Regroupement de documents texte à l'aide de k-means**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/text/plot_document_clustering.ipynb)<br/>([*Clustering text documents using k-means*](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html))

Regroupement de documents à l'aide de MiniBatchKMeans creux.

##### [**Apprentissage en ligne d'un dictionnaire de parties de visages**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_dict_face_patches.ipynb)<br/>([*Online learning of a dictionary of parts of faces*](https://scikit-learn.org/stable/auto_examples/cluster/plot_dict_face_patches.html))


#### Références

[?] D. Sculley [“**Web Scale K-Means clustering**](https://www.eecs.tufts.edu/~dsculley/papers/fastkmeans.pdf)[”](https://drive.google.com/file/d/1urEewuktSLJpV_PhQh7ita5O-VjRLtyw/view?usp=share_link), Proceedings of the 19th international conference on World wide web (2010)



## 2.3.3. **Propagation par affinité**<br/>([*Affinity Propagation*](https://scikit-learn.org/stable/modules/clustering.html#adjusted-rand-score))

[**`AffinityPropagation`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html) crée des clusters en envoyant des messages entre des paires d'échantillons jusqu'à convergence. Un ensemble de données est ensuite décrit à l'aide d'un petit nombre d'exemplaires, qui sont identifiés comme étant les plus représentatifs des autres échantillons. Les messages envoyés entre les paires représentent l'aptitude d'un échantillon à être l'exemplaire de l'autre, qui est mis à jour en réponse aux valeurs des autres paires. Cette mise à jour se produit de manière itérative jusqu'à la convergence, moment auquel les exemplaires finaux sont choisis, et donc le regroupement final est donné.

<figure class="align-center">
<a class="reference external image-reference" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_affinity_propagation.html"><img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_affinity_propagation_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_affinity_propagation_001.png" style="width: 320.0px; height: 240.0px;" /></a>
</figure>

La propagation par affinité peut être intéressante car elle choisit le nombre de clusters en fonction des données fournies. A cet effet, les deux paramètres importants sont la préférence, qui contrôle le nombre d'exemplaires utilisés, et le facteur d'amortissement qui amortit les messages de responsabilité et de disponibilité pour éviter les oscillations numériques lors de la mise à jour de ces messages.

Le principal inconvénient de la propagation par affinité est sa complexité. L'algorithme a une complexité temporelle de l'ordre de $\mathcal{O}(n^2 T)$, où $n$ est le nombre d'échantillons et $T$ est le nombre d'itérations jusqu'à la convergence. De plus, la complexité mémoire est de l'ordre de $\mathcal{O}(n^2)$ si une matrice de similarité dense est utilisée, mais réductible si une matrice de similarité creuse est utilisée. Cela rend la propagation par affinité plus appropriée pour les ensembles de données de petite à moyenne taille.

### Exemples

#### [**Démo de l'algorithme de clustering par propagation d'affinité**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_affinity_propagation.ipynb)<br/>([*Demo of affinity propagation clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_affinity_propagation.html))

Propagation d'affinité sur un jeu de données 2D synthétique avec 3 classes.

#### [**Visualisation de la structure de la bourse**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/applications/plot_stock_market.ipynb)<br/>([*Visualizing the stock market structure*](https://scikit-learn.org/stable/auto_examples/applications/plot_stock_market.html))

Propagation d'affinité sur les séries chronologiques financières pour trouver des groupes d'entreprises.

### Description de l'algorithme

Les messages envoyés entre les points appartiennent à l'une des deux catégories. Le premier est la responsabilité $r(i, k)$, qui est la preuve accumulée que l'échantillon $k$ devrait être le représentant de l'échantillon $i$. La seconde est la disponibilité $a(i, k)$ qui est la preuve accumulée que l'échantillon $i$ devrait choisir l'échantillon $k$ comme son représentant, et considère les valeurs de tous les autres échantillons dont $k$ devrait être le représentant. De cette manière, les représentants sont choisis par des échantillons s'ils sont (1) suffisamment similaires à de nombreux échantillons et (2) choisis par de nombreux échantillons pour être représentatifs d'eux-mêmes.

Plus formellement, la responsabilité d'un échantillon $k$ pour être le représentant de l'échantillon $i$ est donné par :

$$r(i, k) \leftarrow s(i, k) - max [ a(i, k') + s(i, k') \forall k' \neq k ]$$

Où $s(i, k)$ est la similarité entre les échantillons $i$ et $k$. La disponibilité de l'échantillon $k$ pour être le représentant de l'échantillon $i$ est donné par:

$$a(i, k) \leftarrow min [0, r(k, k) + \sum_{i'~s.t.~i' \notin \{i, k\}}{r(i', k)}]$$

Pour commencer, toutes les valeurs de $r$ et $a$ sont mises à zéro, et le calcul de chacun itère jusqu'à convergence. Comme discuté ci-dessus, afin d'éviter les oscillations numériques lors de la mise à jour des messages, le facteur d'amortissement $\lambda$ est introduit au processus d'itération :

$$r_{t+1}(i, k) = \lambda\cdot r_{t}(i, k) + (1-\lambda)\cdot r_{t+1}(i, k)$$
$$a_{t+1}(i, k) = \lambda\cdot a_{t}(i, k) + (1-\lambda)\cdot a_{t+1}(i, k)$$

où $t$ indique les temps d'itération.

## 2.3.4. Décalage moyen

Le clustering [**`MeanShift`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html) vise à découvrir des *blobs* dans une densité lisse d'échantillons. Il s'agit d'un algorithme basé sur le centroïde, qui fonctionne en mettant à jour les candidats pour que les centroïdes soient la moyenne des points dans une région donnée. Ces candidats sont ensuite filtrés dans une étape de post-traitement pour éliminer les quasi-doublons pour former l'ensemble final de centroïdes.

Étant donné un barycentre candidat $x_i$ pour l'itération $t$, le candidat est mis à jour selon l'équation suivante :

$$x_i^{t+1} = m(x_i^t)$$

Où $N(x_i)$ est le voisinage des échantillons à une distance donnée autour de $x_i$ et $m$ est le vecteur de *décalage moyen* qui est calculé pour chaque centroïde qui pointe vers une région d'augmentation maximale de la densité de points. Ceci est calculé à l'aide de l'équation suivante, mettant à jour efficacement un centroïde pour qu'il soit la moyenne des échantillons dans son voisinage :

$$m(x_i) = \frac{\sum_{x_j \in N(x_i)}K(x_j - x_i)x_j}{\sum_{x_j \in N(x_i)}K(x_j - x_i)}$$

L'algorithme définit automatiquement le nombre de clusters, au lieu de s'appuyer sur le paramètre de bande passante `bandwidth`, qui dicte la taille de la région à parcourir. Ce paramètre peut être défini manuellement, mais peut être estimé à l'aide de la fonction `estimate_bandwidth` fournie, qui est appelée si la bande passante n'est pas définie.

L'algorithme n'est pas hautement évolutif (passage à l'échelle), car il nécessite plusieurs recherches du plus proche voisin pendant son exécution. L'algorithme converge nécessairement, mais il cessera d'itérer lorsque le changement de centroïdes est faible.

L'étiquetage d'un nouvel échantillon est effectué en trouvant le centroïde le plus proche pour un échantillon donné.

<figure class="align-center">
<a class="reference external image-reference" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html"><img alt="../_images/sphx_glr_plot_mean_shift_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_mean_shift_001.png" style="width: 320.0px; height: 240.0px;" /></a>
</figure>

### Exemples

#### [**Une démonstration de l'algorithme de clustering par décalage moyen**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_mean_shift.ipynb)<br/>([*A demo of the mean-shift clustering algorithm*](https://scikit-learn.org/stable/auto_examples/cluster/plot_mean_shift.html))

Clustering par décalage moyen sur un jeu de données 2D synthétique avec 3 classes.

### Références

[?] Dorin Comaniciu, Peter Meer, [“**Mean Shift: A robust approach toward feature space analysis**](https://ieeexplore.ieee.org/document/1000236)[”](https://drive.google.com/file/d/1v5NOqj-DIjEZDWrZfN2yIRw_vbwMu0Ea/view?usp=share_link). IEEE Transactions on Pattern Analysis and Machine Intelligence. 2002. pp. 603-619.

## 2.3.5. Regroupement spectral

[**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) effectue une intégration de faible dimension de la matrice d'affinité entre les échantillons, suivie d'un regroupement, par exemple, par KMeans, des composants des vecteurs propres dans l'espace de faible dimension. Il est particulièrement efficace en termes de calcul si la matrice d'affinité est creuse et que le solveur `amg` est utilisé pour le problème des valeurs propres (notez que le solveur `amg` nécessite que le module [**`pyamg`**](https://github.com/pyamg/pyamg) soit installé.)

La version actuelle de SpectralClustering nécessite que le nombre de clusters soit spécifié à l'avance. Cela fonctionne bien pour un petit nombre de clusters, mais n'est pas conseillé pour de nombreux clusters.

Pour deux clusters, SpectralClustering résout une relaxation convexe du problème des [coupes normalisées](https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf) sur le graphe de similarité : coupant le graphe en deux pour que le poids des arêtes coupées soit petit par rapport aux poids des arêtes à l'intérieur de chaque cluster. Ce critère est particulièrement intéressant lorsque l'on travaille sur des images, où les sommets du graphe sont des pixels, et les poids des arêtes du graphe de similarité sont calculés en fonction d'un gradient de l'image.

<img alt="noisy_img" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_segmentation_toy_001.png" style="width: 500.0px; height: 250.0px;" /></a> <a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html"><img alt="segmented_img" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_segmentation_toy_002.png" style="width: 500.0px; height: 250.0px;" />


### Avertissement

Transformer la distance en similitudes bien élevées

Notez que si les valeurs de votre matrice de similarité ne sont pas bien distribuées, par ex. avec des valeurs négatives ou avec une matrice de distance plutôt qu'une similarité, le problème spectral sera singulier et le problème non résoluble. Dans ce cas il est conseillé d'appliquer une transformation sur les entrées de la matrice. Par exemple, dans le cas d'une matrice de distance signée, il est courant d'appliquer un noyau de chaleur :

    similarité = np.exp(-beta * distance / distance.std())

Voir les exemples d'une telle application.

### Exemples

#### [**Regroupement spectral pour la segmentation d'images**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_segmentation_toy.ipynb)<br/>([*Spectral clustering for image segmentation*](https://scikit-learn.org/stable/auto_examples/cluster/plot_segmentation_toy.html))

Segmentation d'objets à partir d'un arrière-plan bruyant à l'aide du regroupement spectral.

#### [**Segmenter en régions l'image des pièces de monnaie grecques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_segmentation.ipynb)<br/>([*Segmenting the picture of greek coins in regions*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html))

Regroupement spectral pour diviser l'image des pièces de monnaie en régions.

#### 2.3.5.1. Différentes stratégies d'attribution d'étiquettes

Différentes stratégies d'attribution d'étiquettes peuvent être utilisées, correspondant au paramètre `assign_labels` de [**`SpectralClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html). La stratégie `"kmeans"` peut correspondre à des détails plus fins, mais peut être instable. En particulier, à moins que vous ne contrôliez le `random_state`, il peut ne pas être reproductible d'une exécution à l'autre, car il dépend de l'initialisation aléatoire. La stratégie alternative de `"discretize"` est reproductible à 100%, mais tend à créer des parcelles de forme assez régulière et géométrique. L'option `"cluster_qr"` récemment ajoutée est une alternative déterministe qui tend à créer le meilleur partitionnement visuel sur l'exemple d'application ci-dessous.

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;kmeans&quot;</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;discretize&quot;</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">assign_labels=&quot;cluster_qr&quot;</span></code></p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_kmeans" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_001.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
<td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_discretize" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_002.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
<td><p><a class="reference external" href="https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html"><img alt="coin_cluster_qr" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_coin_segmentation_003.png" style="width: 175.0px; height: 175.0px;" /></a></p></td>
</tr>
</tbody>
</table>

##### Références

[?] Stella X. Yu, Jianbo Shi, [“**Multiclass spectral clustering**](http://lear.inrialpes.fr/people/triggs/events/iccv03/cdrom/iccv03/0313_yu.pdf)[”](https://drive.google.com/file/d/1JsQ_aa3BheGcipQONMcdZa0AzISuGJ-C/view?usp=share_link), 2003

[?] Anil Damle, Victor Minden, Lexing Ying, [“**Simple, direct, and efficient multi-way spectral clustering**](https://doi.org/10.1093/imaiai/iay008)[”](https://drive.google.com/file/d/1eWIeV12C78cthlB7D498g3pq_QvtCofI/view?usp=share_link), 2019

#### 2.3.5.2. Graphiques de regroupement spectral

Le clustering spectral peut également être utilisé pour partitionner des graphiques via leurs intégrations spectrales. Dans ce cas, la matrice d'affinité est la matrice d'adjacence du graphe, et SpectralClustering est initialisé avec `affinity='precomputed'` :

In [ ]:
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(3, affinity='precomputed', n_init=100,
                        assign_labels='discretize')
sc.fit_predict(adjacency_matrix)

##### Références



[?] Ulrike von Luxburg,  [“**A Tutorial on Spectral Clustering**](https://arxiv.org/pdf/0711.0189.pdf)[”](https://drive.google.com/file/d/1Hb7HwNENVUXgosAk3bG0tpcM-AhfjQdF/view?usp=share_link), 2007

[?] Jianbo Shi, Jitendra Malik, [“**Normalized cuts and image segmentation**](https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf)[”](https://drive.google.com/file/d/1-_p-J8SktpTVCdTs140WRNpJ5qPFHbzD/view?usp=share_link), 2000

[?] Marina Meila, Jianbo Shi, [“**A Random Walks View of Spectral Segmentation**](https://www.ri.cmu.edu/pub_files/pub3/maila_marina_2001_2/maila_marina_2001_2.pdf)[”](https://drive.google.com/file/d/19zsTu3Vz1NtibSD2qw_3u44uZYvsj3na/view?usp=share_link), 2001

[?] Andrew Y. Ng, Michael I. Jordan, Yair Weiss, [“**On Spectral Clustering: Analysis and an algorithm**](https://proceedings.neurips.cc/paper/2001/file/801272ee79cfde7fa5960571fee36b9b-Paper.pdf)[”](https://drive.google.com/file/d/1DNbfVClFKFNdZ7ubynCUbGSvPdtUHRJG/view?usp=share_link), 2001

[?] David Zhuzhunashvili, Andrew Knyazev, [“**Preconditioned Spectral Clustering for Stochastic Block Partition Streaming Graph Challenge**](https://arxiv.org/pdf/1708.07481.pdf)[”](https://drive.google.com/file/d/1QhXpbu371LnDwzTeIpidRr-YCWRA-ZJE/view?usp=share_link)



## 2.3.6. Classification hiérarchique

Le clustering hiérarchique est une famille générale d'algorithmes de clustering qui construisent des clusters imbriqués en les fusionnant ou en les divisant successivement. Cette hiérarchie de clusters est représentée sous forme d'arbre (ou dendrogramme). La racine de l'arbre est la grappe unique qui rassemble tous les échantillons, les feuilles étant les grappes à un seul échantillon. Voir la [**page Wikipédia**](https://en.wikipedia.org/wiki/Hierarchical_clustering) pour plus de détails.

L'objet [**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) effectue un clustering hiérarchique en utilisant une approche ascendante : chaque observation commence dans son propre cluster, et les clusters sont successivement fusionnés. Les critères de liaison déterminent la métrique utilisée pour la stratégie de fusion :

* **Ward** minimise la somme des différences au carré dans tous les clusters. Il s'agit d'une approche de minimisation de la variance et, en ce sens, elle est similaire à la fonction objectif k-means mais abordée avec une approche hiérarchique agglomérative.
* **La liaison maximale** ou **complète** minimise la distance maximale entre les observations de paires de grappes.
* **Le couplage moyen** minimise la moyenne des distances entre toutes les observations de paires de grappes.
* **Le couplage unique** minimise la distance entre les observations les plus proches des paires de grappes.

L'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) peut également évoluer vers un grand nombre d'échantillons lorsqu'il est utilisé conjointement avec une matrice de connectivité, mais est coûteux en calcul lorsqu'aucune contrainte de connectivité n'est ajoutée entre les échantillons : il considère à chaque étape toutes les fusions possibles.

[**`FeatureAgglomeration`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)

Le [**`FeatureAgglomeration`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) utilise le clustering agglomératif pour regrouper les caractéristiques qui semblent très similaires, réduisant ainsi le nombre de caractéristiques. C'est un outil de réduction de dimensionnalité, voir [**Réduction de dimensionnalité non supervisée** (6.5)](https://scikit-learn.org/stable/modules/unsupervised_reduction.html#data-reduction).

### 2.3.6.1. Différents types de liaison : Ward, complète, moyenne et simple liaison

L'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) prend en charge les stratégies de liaison Ward, unique, moyenne et complète.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_linkage_comparison_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_linkage_comparison_001.png" style="width: 589.1px; height: 623.5px;" />

Le cluster agglomératif a un comportement « riche devient plus riche » qui conduit à des tailles de cluster inégales. À cet égard, la liaison simple est la pire stratégie, et Ward donne les tailles les plus régulières. Cependant, l'affinité (ou la distance utilisée dans le clustering) ne peut pas être modifiée avec Ward, donc pour les métriques non euclidiennes, la liaison moyenne est une bonne alternative. La liaison unique, bien qu'elle ne soit pas robuste aux données bruitées, peut être calculée très efficacement et peut donc être utile pour fournir un regroupement hiérarchique d'ensembles de données plus volumineux. La liaison simple peut également bien fonctionner sur des données non globulaires.

#### Exemples

##### [**Divers clustering agglomératifs sur un plongement 2D de chiffres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_digits_linkage.ipynb)<br/>([*Various Agglomerative Clustering on a 2D embedding of digits*](https://scikit-learn.org/stable/auto_examples/cluster/plot_digits_linkage.html))

Exploration des différentes stratégies de liaison dans un jeu de données réel.

### 2.3.6.2. Visualisation de la hiérarchie des clusters

Il est possible de visualiser l'arbre représentant la fusion hiérarchique des clusters sous forme de dendrogramme. L'inspection visuelle peut souvent être utile pour comprendre la structure des données, mais plus encore dans le cas d'échantillons de petite taille.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_dendrogram_001.png)

### 2.3.6.3. Ajout de contraintes de connectivité

Un aspect intéressant d'[**`AgglomerativeClustering`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) est que des contraintes de connectivité peuvent être ajoutées à cet algorithme (seuls les clusters adjacents peuvent être fusionnés), via une matrice de connectivité qui définit pour chaque échantillon les échantillons voisins suivant une structure particulière des données. Par exemple, dans l'exemple du rouleau suisse ci-dessous, les contraintes de connectivité interdisent la fusion de points qui ne sont pas adjacents sur le rouleau suisse, et évitent ainsi la formation de clusters qui s'étendent sur des plis superposés du rouleau.

<img alt="unstructured" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_001.png" style="width: 313.6px; height: 235.2px;" /><img alt="structured" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_002.png" style="width: 313.6px; height: 235.2px;" />

Ces contraintes sont utiles pour imposer une certaine structure locale, mais elles rendent aussi l'algorithme plus rapide, surtout lorsque le nombre d'échantillons est élevé.

Les contraintes de connectivité sont imposées via une matrice de connectivité : une matrice creuse scipy qui n'a d'éléments qu'à l'intersection d'une ligne et d'une colonne avec des indices de l'ensemble de données qui doivent être connectés. Cette matrice peut être construite à partir d'informations a priori : par exemple, vous pouvez regrouper des pages Web en ne fusionnant que des pages avec un lien pointant de l'une vers l'autre. Il peut également être appris à partir des données, par exemple en utilisant [**`sklearn.neighbors.kneighbors_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html) pour limiter la fusion aux voisins les plus proches comme dans [**cet exemple**](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering.html), ou en utilisant [**`sklearn.feature_extraction.image.grid_to_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.image.grid_to_graph.html) pour activer uniquement la fusion des pixels voisins sur une image, comme dans l'[**exemple de la pièce**](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html).

#### Exemples

#### [**Une démo de clustering hiérarchique Ward structuré sur une image de pièces**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_ward_segmentation.ipynb)<br/>([*A demo of structured Ward hierarchical clustering on an image of coins*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html))

Ward clustering pour diviser l'image des pièces en régions.

#### [**Clustering hiérarchique : ward structuré vs non structuré**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_coin_ward_segmentation.ipynb)<br/>([*Hierarchical clustering: structured vs unstructured ward*](https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_ward_segmentation.html))

Exemple d'algorithme de Ward sur un rouleau suisse, comparaison des approches structurées versus des approches non structurées.

#### [**Agglomération de caractéristiques vs sélection univariée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_feature_agglomeration_vs_univariate_selection.ipynb)<br/>([*Feature agglomeration vs. univariate selection*](https://scikit-learn.org/stable/auto_examples/cluster/plot_feature_agglomeration_vs_univariate_selection.html))

Exemple de réduction de dimensionnalité avec agglomération de caractéristiques basée sur le regroupement hiérarchique de Ward.

#### [**Agglomération avec et sans structure**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering.ipynb)<br/>([*Agglomerative clustering with and without structure*](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering.html))

#### Avertissement

**Contraintes de connectivité avec liaison simple, moyenne et complète**

Les contraintes de connectivité et les liaisons simples, complètes ou moyennes peuvent améliorer l'aspect 'de plus en plus riche' du clustering agglomératif, en particulier s'ils sont construits avec [**`sklearn.neighbors.kneighbors_graph`**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.kneighbors_graph.html). Dans la limite d'un petit nombre d'amas, ils tendent à donner quelques amas macroscopiquement occupés et presque vides. (voir la discussion dans [**Agglomération avec et sans structure**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering.ipynb)). La liaison unique est l'option de liaison la plus fragile en ce qui concerne cette question.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_001.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_001.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_002.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_002.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_003.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_003.png" style="width: 380.0px; height: 152.0px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_004.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_004.png" style="width: 380.0px; height: 152.0px;" />


### 2.3.6.4. Varier la métrique

Une liaison simple, moyenne et complète peut être utilisée avec une variété de distances (ou affinités), en particulier la distance euclidienne ($\ell_2$), la distance de Manhattan (ou Cityblock, ou $\ell_1$), la distance cosinus ou toute matrice d'affinité précalculée.
* La distance $\ell_1$ est souvent bonne pour les caractéristiques creux ou le bruit creux: c'est-à-dire que de nombreuses caractéristiques sont nulles, comme dans l'exploration de texte utilisant des occurrences de mots rares.
* La distance cosinus est intéressante car elle est invariante aux échelles globales du signal.

Les directives pour choisir une métrique sont d'en utiliser une qui maximise la distance entre les échantillons dans différentes classes et la minimise dans chaque classe.

<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_005.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_005.png" style="width: 204.8px; height: 153.6px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_006.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_006.png" style="width: 204.8px; height: 153.6px;" />
<img alt="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_007.png" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_007.png" style="width: 204.8px; height: 153.6px;" />

#### Exemples

##### [**Clustering agglomératif avec différentes métriques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/2_3_cluster/plot_agglomerative_clustering_metrics.ipynb)<br/>([*Agglomerative clustering with different metrics*](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering_metrics.html))


### 2.3.6.5. Bisecting K-Means

Le [**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) est une variante itérative de [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), utilisant le clustering hiérarchique de division. Au lieu de créer tous les centroïdes en même temps, les centroïdes sont sélectionnés progressivement en fonction d'un clustering précédent : un cluster est divisé en deux nouveaux clusters à plusieurs reprises jusqu'à ce que le nombre cible de clusters soit atteint.

[**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) est plus efficace que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) lorsque le nombre de clusters est grand car il ne fonctionne que sur un sous-ensemble de données à chaque bissection tandis que [**`KMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) fonctionne toujours sur l'ensemble de données entier.

Bien que [**`BisectingKMeans`**](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.BisectingKMeans.html) ne puisse pas bénéficier des avantages de l'initialisation `"k-means++"` par conception, il produira toujours des résultats comparables à `KMeans(init="k-means++")` en termes d'inertie à des coûts de calcul moins chers, et produira probablement de meilleurs résultats que `KMeans` avec une initialisation aléatoire.

Cette variante est plus efficace pour le clustering agglomératif si le nombre de clusters est petit par rapport au nombre de points de données.

Cette variante ne produit pas non plus de clusters vides.

**Il existe deux stratégies pour sélectionner le cluster à scinder :**
* `bisecting_strategy="largest_cluster"` sélectionne le cluster ayant le plus de points
* `bisecting_strategy="biggest_inertia"` sélectionne le cluster avec la plus grande inertie (cluster avec la plus grande somme des erreurs au carré à l'intérieur)

La sélection par la plus grande quantité de points de données dans la plupart des cas produit un résultat aussi précis que la sélection par inertie et est plus rapide (en particulier pour une plus grande quantité de points de données, où l'erreur de calcul peut être coûteuse).

La sélection par la plus grande quantité de points de données produira également probablement des clusters de tailles similaires, tandis que `KMeans` est connu pour produire des clusters de tailles différentes.

La différence entre Bisecting K-Means et Regular K-Means peut être vue sur l'exemple [**Bisecting K-Means et Regular K-Means Comparaison des performances**](https://scikit-learn.org/stable/auto_examples/cluster/plot_bisect_kmeans.html). Alors que l'algorithme K-Means régulier a tendance à créer des clusters non liés, les clusters de Bisecting K-Means sont bien ordonnés et créent une hiérarchie assez visible.

#### Références

[?] Michael Steinbach, George Karypis and Vipin Kumar, [“**A Comparison of Document Clustering Techniques**](http://www.philippe-fournier-viger.com/spmf/bisectingkmeans.pdf)[”](https://drive.google.com/file/d/1XwuyijuSlSEzdhS3UUzxfWB-K16YLoMs/view?usp=share_link), Department of Computer Science and Egineering, University of Minnesota (June 2000)

[?] K.Abirami and Dr.P.Mayilvahanan, [“**Performance Analysis of K-Means and Bisecting K-Means Algorithms in Weblog Data**](https://ijeter.everscience.org/Manuscripts/Volume-4/Issue-8/Vol-4-issue-8-M-23.pdf)[”](https://drive.google.com/file/d/14ypP25t0s7Oxqg15aDqjF0T5fatoz1SM/view?usp=share_link), International Journal of Emerging Technologies in Engineering Research (IJETER) Volume 4, Issue 8, (August 2016)

[?] Jian Di, Xinyue Gou, [“**Bisecting K-means Algorithm Based on K-valued Self-determining and Clustering Center Optimization**](http://www.jcomputers.us/vol13/jcp1306-01.pdf)[”](https://drive.google.com/file/d/1pTCDwE70kjr1y7GmIYfJNatHGhU58tQe/view?usp=share_link), School of Control and Computer Engineering, North China Electric Power University, Baoding, Hebei, China (August 2017)
